In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.sparse as sps
from statsmodels.api import Logit
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split
from sklearn.metrics import classification_report, precision_recall_curve, roc_curve, confusion_matrix

from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
import sklearn

Here the results from 'Adult EDA" file are going to be used

In [2]:
# %run "Adult EDA.ipynb"

In [96]:
adult_columns = [
    "Age",
    "Workclass",
    "final weight",
    "Education",
    "Education-Num",
    "Marital Status",
    "Occupation",
    "Relationship",
    "Ethnic group",
    "Sex",
    "Capital Gain",
    "Capital Loss",
    "Hours per week",
    "Country",
    "Income",
]

df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", 
                 header = None, names = adult_columns)
df = df.replace(to_replace= ' ?', value = np.nan)

In [97]:
df = df.drop(['Education-Num'], axis = 'columns')

In [98]:
categorical_features_list = ['Workclass', 'Marital Status', 'Occupation', 
                             'Relationship', 'Ethnic group', 'Sex', 'Country']

In [99]:
# for col in categorical_features_list:
#     print(f'{col}\n{df[col].value_counts()}\n')

As in the whole dataset (X) we have only one 'Holand-Netherlands' value in 'Country' column, we have to process it separately, because in case if it appears in the test set, model will not be able to predict target for such a record. For the initial model, where there are no changes in data, this observation will be removed

In [100]:
print(df.shape)
df_no_nl = df.copy()
df_no_nl.drop(df_no_nl.loc[df['Country']==' Holand-Netherlands'].index, inplace=True)
print(df_no_nl.shape)

(32561, 14)
(32560, 14)


In [101]:
X = df_no_nl.drop(['Income'], axis = 'columns')
y = df_no_nl['Income']

In [102]:
X, X_test, y, y_test = train_test_split(X, y, test_size = 0.2)

## 1. Features preprocessing

First, all variables have to be transformed to numerical format to feed them to LogisticRegression function:

In [103]:
X_train = X.copy()
y_train = y.copy()

data_train = pd.merge(left=y_train, right=X_train, left_index=True, right_index=True)
data_train.shape

(26048, 14)

In [104]:
data_train.head()

,Income,Age,Workclass,final weight,Education,Marital Status,Occupation,Relationship,Ethnic group,Sex,Capital Gain,Capital Loss,Hours per week,Country
23504,>50K,31,Local-gov,153005,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States
4116,<=50K,65,Private,361721,Assoc-voc,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,20,United-States
7027,<=50K,64,NaN,143716,Masters,Married-civ-spouse,NaN,Husband,White,Male,0,0,2,United-States
12918,>50K,61,NaN,198686,Assoc-acdm,Married-civ-spouse,NaN,Husband,White,Male,0,0,2,United-States
31261,<=50K,24,Private,222853,Some-college,Never-married,Craft-repair,Unmarried,White,Male,0,0,50,United-States


In this dataset we have only one feature, where the order matters - Education, so it will be transformed with using OrdinalEncoder. For all the rest of categorical features the order does not matter, hense we can apply OneHotEncoder() to them.

# 1st model 
### Inital model without changes in data

In [10]:
numerical_features_list = ['Age', 'final weight', 'Capital Gain', 'Capital Loss', 'Hours per week']
categorical_features_list = ['Workclass', 'Marital Status', 'Occupation', 
                             'Relationship', 'Ethnic group', 'Sex', 'Country']

In [105]:
# A function, that performs all needed data preparation and feeds it to LogisticRegression

def get_LR_performance(data, numerical_features_list: list, categorical_features_list:list):
    """The function performs data preprocessing, creates pipeline with LogisticRegression model, 
        and prints it's performance out

    Args:
        data: full dataset with features and target
        numerical_features_list (list): list of features, that have to be processed by Standard scaler
        categorical_features_list (list): list of features, that have to be processed by OneHotEncoder
    """

    X = data.drop(columns=['Income'])
    y = data["Income"]

    columntransformer = ColumnTransformer(transformers = [
    ('ordinal', OrdinalEncoder(categories=[[' Preschool',' 1st-4th',' 5th-6th',' 7th-8th',' 9th',' 10th',' 11th',
                                      ' 12th',' HS-grad',' Some-college',' Assoc-voc',' Assoc-acdm', 
                                      ' Bachelors',' Masters',' Prof-school',' Doctorate']]),
                                make_column_selector(pattern = 'Education')),
    ('stand scaler', StandardScaler(), numerical_features_list),
    ('onehot', OneHotEncoder(dtype='int', drop='first'), categorical_features_list)],
    remainder='drop')
    
    pipe = make_pipeline(columntransformer, LogisticRegression(max_iter=10000)).fit(X, y)

    y_pred = pipe.predict(X)
    
    scores = cross_val_score(pipe, X, y, cv=5, scoring='f1_macro')
    f1_mean_score = round(np.mean(scores),2)
    f1_std = round(np.std(scores),2)
    
    report = classification_report(y, y_pred, target_names=data['Income'].unique())
   
    print(f'f1 score: mean = {f1_mean_score} | std = {f1_std}')
    print(report)


In [106]:
get_LR_performance(data_train, numerical_features_list, categorical_features_list)

f1 score: mean = 0.78 | std = 0.0
              precision    recall  f1-score   support

        >50K       0.88      0.93      0.90     19715
       <=50K       0.73      0.60      0.66      6333

    accuracy                           0.85     26048
   macro avg       0.81      0.76      0.78     26048
weighted avg       0.84      0.85      0.84     26048



### Let's now understand significance of features with the help of Logit() function from statsmodel

In [107]:
def logit_summary(data, numerical_features: list, categorical_features: list):
    """Function performs data preprocessing and applies Logit() function. After that retuns summary which contains featues significances

    Args:
        X (Series object): X_train DataFrame of features
        y (array): y_train - target
        numerical_features_list (list): list of features, that have to be processed by Standard scaler
        categorical_features_list (list): list of features, that have to be processed by OneHotEncoder

    Returns:
        Summary: summary of statsmodel Logit() model with the help of which the decision about 
                keeping or modifying/removing a feature can be made
    """

    X = data.drop(columns=['Income'])
    y = data["Income"]

    column_transformer = ColumnTransformer(transformers = [
        ('ordinal', OrdinalEncoder(categories=[[' Preschool',' 1st-4th',' 5th-6th',' 7th-8th',' 9th',' 10th',' 11th',
                                          ' 12th',' HS-grad',' Some-college',' Assoc-voc',' Assoc-acdm', 
                                          ' Bachelors',' Masters',' Prof-school',' Doctorate']]),
         make_column_selector(pattern = 'Education')),
        ('stand_scaler', StandardScaler(), numerical_features),
        ('onehot', OneHotEncoder(dtype='int', drop='first'), categorical_features)],
        remainder='drop')
    
    X_trans = column_transformer.fit_transform(X)
    
    if sps.issparse(X_trans):
        X_trans = X_trans.toarray()
        
    x_columns_names = column_transformer.get_feature_names_out()
    X_trans = pd.DataFrame(X_trans, columns = x_columns_names)
    
    y_train_df = pd.DataFrame(y)
    onehot = OneHotEncoder(dtype='int', drop='first')
    y_trans = onehot.fit_transform(y_train_df)
    y_column_name = onehot.get_feature_names_out()
    y_trans = pd.DataFrame.sparse.from_spmatrix(y_trans, columns=y_column_name)
    
    model = Logit(y_trans, X_trans).fit_regularized()
    summary = model.summary()
    
    return summary

In [108]:
summary = logit_summary(data_train, numerical_features_list, categorical_features_list)
summary

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.3249190830739871
            Iterations: 564
            Function evaluations: 566
            Gradient evaluations: 564


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           Income_ >50K   No. Observations:                26048
Model:                          Logit   Df Residuals:                    25966
Method:                           MLE   Df Model:                           81
Date:                Thu, 27 Apr 2023   Pseudo R-squ.:                  0.4142
Time:                        07:53:43   Log-Likelihood:                -8463.5
converged:                       True   LL-Null:                       -14448.
Covariance Type:            nonrobust   LLR p-value:                     0.000
=================================================================================================================
                                                    coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
ordinal__Education                                0.2805      0.010     27.551      0.000       0.261       0.300
stand_scaler__Age                                 0.2866      0.024     11.929      0.000       0.240       0.334
stand_scaler__Capital Gain                        2.2530      0.082     27.367      0.000       2.092       2.414
stand_scaler__Capital Loss                        0.2658      0.017     16.019      0.000       0.233       0.298
onehot__Workclass_ Local-gov                     -0.7254      0.124     -5.861      0.000      -0.968      -0.483
onehot__Workclass_ Never-worked                  -1.3907   1.12e+07  -1.24e-07      1.000    -2.2e+07     2.2e+07
onehot__Workclass_ Private                       -0.5207      0.103     -5.039      0.000      -0.723      -0.318
onehot__Workclass_ Self-emp-inc                  -0.2213      0.135     -1.636      0.102      -0.486       0.044
onehot__Workclass_ Self-emp-not-inc              -0.9669      0.120     -8.042      0.000      -1.203      -0.731
onehot__Workclass_ State-gov                     -0.9210      0.137     -6.716      0.000      -1.190      -0.652
onehot__Workclass_ Without-pay                  -11.6559    118.451     -0.098      0.922    -243.815     220.503
onehot__Workclass_nan                             0.6772   1.12e+07   6.05e-08      1.000    -2.2e+07     2.2e+07
onehot__Marital Status_ Married-AF-spouse         1.9123      0.603      3.172      0.002       0.731       3.094
onehot__Marital Status_ Married-civ-spouse        1.3535      0.295      4.590      0.000       0.775       1.931
onehot__Marital Status_ Married-spouse-absent    -0.1584      0.259     -0.611      0.541      -0.667       0.350
onehot__Marital Status_ Never-married            -0.6020      0.095     -6.317      0.000      -0.789      -0.415
onehot__Marital Status_ Separated                -0.1652      0.180     -0.916      0.360      -0.519       0.188
onehot__Marital Status_ Widowed                  -0.1487      0.173     -0.861      0.389      -0.487       0.190
onehot__Occupation_ Armed-Forces                 -2.3624      5.586     -0.423      0.672     -13.311       8.586
onehot__Occupation_ Craft-repair                  0.0934      0.088      1.057      0.290      -0.080       0.266
onehot__Occupation_ Exec-managerial               0.9098      0.085     10.721      0.000       0.743       1.076
onehot__Occupation_ Farming-fishing              -0.7032      0.152     -4.629      0.000      -1.001      -0.405
onehot__Occupation_ Handlers-cleaners            -0.6907      0.159     -4.337      0.000      -1.003      -0.379
onehot__Occupation_ Machine-op-inspct            -0.2306      0.112     -2.063      0.039      -0.450      -0.012
onehot__Occupation_ Other-service                -0.7967      0.128     -6.231      0.000      -1.047      -0.546
onehot__Occupation_ Priv-house-serv              

# 2nd model
### Same model, but without 'final weight'

As we remember from EDA, **'final weight'** feature did not pass the significance border. Let's try to remove it and check the performance

In [15]:
numerical_features_list = ['Age', 'Capital Gain', 'Capital Loss', 'Hours per week']
categorical_features_list = ['Workclass', 'Marital Status', 'Occupation', 
                             'Relationship', 'Ethnic group', 'Sex', 'Country']

In [16]:
get_LR_performance(data_train, numerical_features_list, categorical_features_list)

f1 score: mean = 0.78 | std = 0.01
              precision    recall  f1-score   support

        >50K       0.88      0.93      0.91     19769
       <=50K       0.74      0.61      0.67      6279

    accuracy                           0.85     26048
   macro avg       0.81      0.77      0.79     26048
weighted avg       0.85      0.85      0.85     26048



#### Performance in generel has not changed, let's check if features' significances have changed

In [17]:
summary = logit_summary(data_train, numerical_features_list, categorical_features_list)
summary

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.31664490661714106
            Iterations: 593
            Function evaluations: 595
            Gradient evaluations: 593


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           Income_ >50K   No. Observations:                26048
Model:                          Logit   Df Residuals:                    25965
Method:                           MLE   Df Model:                           82
Date:                Thu, 27 Apr 2023   Pseudo R-squ.:                  0.4267
Time:                        07:20:14   Log-Likelihood:                -8248.0
converged:                       True   LL-Null:                       -14386.
Covariance Type:            nonrobust   LLR p-value:                     0.000
=================================================================================================================
                                                    coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
ordinal__Education                                0.2640      0.010     25.712      0.000       0.244       0.284
stand_scaler__Age                                 0.3472      0.025     13.987      0.000       0.299       0.396
stand_scaler__Capital Gain                        2.3581      0.085     27.903      0.000       2.192       2.524
stand_scaler__Capital Loss                        0.2582      0.017     15.468      0.000       0.225       0.291
stand_scaler__Hours per week                      0.3782      0.022     16.924      0.000       0.334       0.422
onehot__Workclass_ Local-gov                     -0.8585      0.126     -6.832      0.000      -1.105      -0.612
onehot__Workclass_ Never-worked                  -1.5254    8.7e+13  -1.75e-14      1.000   -1.71e+14    1.71e+14
onehot__Workclass_ Private                       -0.6599      0.104     -6.340      0.000      -0.864      -0.456
onehot__Workclass_ Self-emp-inc                  -0.3947      0.137     -2.885      0.004      -0.663      -0.127
onehot__Workclass_ Self-emp-not-inc              -1.1917      0.122     -9.774      0.000      -1.431      -0.953
onehot__Workclass_ State-gov                     -1.0396      0.140     -7.439      0.000      -1.314      -0.766
onehot__Workclass_ Without-pay                  -10.4699     80.189     -0.131      0.896    -167.638     146.699
onehot__Workclass_nan                             0.8925    8.7e+13   1.03e-14      1.000   -1.71e+14    1.71e+14
onehot__Marital Status_ Married-AF-spouse         1.9581      0.551      3.554      0.000       0.878       3.038
onehot__Marital Status_ Married-civ-spouse        1.6936      0.218      7.760      0.000       1.266       2.121
onehot__Marital Status_ Married-spouse-absent     0.0305      0.256      0.119      0.905      -0.471       0.532
onehot__Marital Status_ Never-married            -0.5030      0.096     -5.226      0.000      -0.692      -0.314
onehot__Marital Status_ Separated                -0.0975      0.179     -0.544      0.586      -0.449       0.254
onehot__Marital Status_ Widowed                  -0.0011      0.171     -0.007      0.995      -0.337       0.335
onehot__Occupation_ Armed-Forces                 -1.1028      1.517     -0.727      0.467      -4.076       1.871
onehot__Occupation_ Craft-repair                  0.0784      0.089      0.886      0.376      -0.095       0.252
onehot__Occupation_ Exec-managerial               0.8091      0.086      9.454      0.000       0.641       0.977
onehot__Occupation_ Farming-fishing              -1.0803      0.160     -6.745      0.000      -1.394      -0.766
onehot__Occupation_ Handlers-cleaners            -0.6982      0.160     -4.367      0.000      -1.011      -0.385
onehot__Occupation_ Machine-op-inspct            -0.3785      0.115     -3.299      0.001      -0.603      -0.154
onehot__Occupation_ Other-service                

According to Logit() results, all of numerical features are statistically significant. Some categoties in a couple of categotical features have to be clustered as they are insignificant. 

Assumption 1. Workclasses representatives, that do not work or work without pay will have less than 50k, so can become one cluster.

Assumption 2. Single people tend to earn more, as they have more free time for career development; so values of Marital Status feature can be clustered to Sigle and Married 

Assumption 3. Occupation has no impact on Income, as all categories are insignificant, so could be removed from the model. But before, they will be left like this, as from the EDA we saw that this feature is significant

Assumption 4. All categories of Relationship, Ethnic Group and Sex features are significant.

Assumption 5. Most of countries have no impact on target, it's possible to cluster them to developed and developing. 

# 3rd model
### Clustering categories of features


In [109]:
categorical_features_list = ['Workclass', 'Marital Status', 'Occupation', 
                             'Relationship', 'Ethnic group', 'Sex', 'Country']

In [114]:
data_clustered = data_train.copy()

def cluster_categorical(data):

    # cluster Workclass
    data['Workclass'] = data['Workclass'].replace({' Never-worked': ' Without-pay'})

    # cluster Marital status
    data.loc[
        lambda x: x["Marital Status"].isin([' Widowed', ' Separated', ' Married-spouse-absent', ' Never-married', ' Divorced']), "Marital Status"
    ] = "Single"

    data.loc[
        lambda x: x["Marital Status"].isin([' Married-AF-spouse', ' Married-civ-spouse']), "Marital Status"
    ] = "Married"

     # cluster Relationship
    data.loc[
        lambda x: x["Relationship"].isin([' Husband', ' Wife', ' Own-child']), "Relationship"
    ] = "Family"

    data.loc[
        lambda x: x["Relationship"].isin([' Not-in-family', ' Unmarried']), "Relationship"
    ] = "Not-in-Family"

    # cluster Countries
    data.loc[
        lambda x: x["Country"].isin([' Holand-Netherlands', ' Scotland', ' Italy', ' England', ' Ireland', ' Germany', ' Hong',  ' France', ' Taiwan', 
                                    ' Japan', ' Puerto-Rico', ' Canada', ' United-States']), "Country"
    ] = "Developed"

    data.loc[
        lambda x: x["Country"].isin([' Hungary', ' Greece', ' Portugal', ' Poland', ' Yugoslavia', ' Cambodia', ' Iran',  ' Philippines', ' Laos', ' Thailand', ' Vietnam', ' South', 
                                    ' China', ' India', ' Honduras', ' Outlying-US(Guam-USVI-etc)', ' Trinadad&Tobago', ' Ecuador',  ' Philippines', ' Nicaragua',
                                    ' Peru', ' Haiti', ' Columbia', ' Guatemala', ' Dominican-Republic', ' Jamaica',  ' Cuba', ' El-Salvador', ' Mexico']), "Country"
    ] = "Developing"

    return data

data_clustered = cluster_categorical(data_clustered)

In [115]:
data_clustered.head()

,Income,Age,Workclass,final weight,Education,Marital Status,Occupation,Relationship,Ethnic group,Sex,Capital Gain,Capital Loss,Hours per week,Country
23504,>50K,31,Local-gov,153005,Bachelors,Married,Prof-specialty,Family,White,Male,0,0,40,Developed
4116,<=50K,65,Private,361721,Assoc-voc,Married,Transport-moving,Family,White,Male,0,0,20,Developed
7027,<=50K,64,NaN,143716,Masters,Married,NaN,Family,White,Male,0,0,2,Developed
12918,>50K,61,NaN,198686,Assoc-acdm,Married,NaN,Family,White,Male,0,0,2,Developed
31261,<=50K,24,Private,222853,Some-college,Single,Craft-repair,Not-in-Family,White,Male,0,0,50,Developed


Let's now apply the pipeline to updated dataset

In [116]:
get_LR_performance(data_clustered, numerical_features_list, categorical_features_list)

f1 score: mean = 0.78 | std = 0.0
              precision    recall  f1-score   support

        >50K       0.88      0.93      0.90     19715
       <=50K       0.73      0.60      0.66      6333

    accuracy                           0.85     26048
   macro avg       0.81      0.76      0.78     26048
weighted avg       0.84      0.85      0.84     26048



In [117]:
summary = logit_summary(data_clustered, numerical_features_list, categorical_features_list)
summary

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.33102155820190604
            Iterations: 285
            Function evaluations: 287
            Gradient evaluations: 285


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           Income_ >50K   No. Observations:                26048
Model:                          Logit   Df Residuals:                    26013
Method:                           MLE   Df Model:                           34
Date:                Thu, 27 Apr 2023   Pseudo R-squ.:                  0.4032
Time:                        07:54:48   Log-Likelihood:                -8622.4
converged:                       True   LL-Null:                       -14448.
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================================
                                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
ordinal__Education                           0.2538      0.010     26.555      0.000       0.235       0.273
stand_scaler__Age                            0.2960      0.022     13.177      0.000       0.252       0.340
stand_scaler__Capital Gain                   2.2571      0.082     27.657      0.000       2.097       2.417
stand_scaler__Capital Loss                   0.2722      0.016     16.522      0.000       0.240       0.304
onehot__Workclass_ Local-gov                -0.9784      0.118     -8.290      0.000      -1.210      -0.747
onehot__Workclass_ Private                  -0.7765      0.097     -7.990      0.000      -0.967      -0.586
onehot__Workclass_ Self-emp-inc             -0.4446      0.131     -3.384      0.001      -0.702      -0.187
onehot__Workclass_ Self-emp-not-inc         -1.2112      0.115    -10.503      0.000      -1.437      -0.985
onehot__Workclass_ State-gov                -1.1778      0.132     -8.899      0.000      -1.437      -0.918
onehot__Workclass_ Without-pay             -24.1164   5.71e+04     -0.000      1.000   -1.12e+05    1.12e+05
onehot__Workclass_nan                       -0.1524   5.47e+05  -2.78e-07      1.000   -1.07e+06    1.07e+06
onehot__Marital Status_Single               -3.7806      0.145    -26.117      0.000      -4.064      -3.497
onehot__Occupation_ Armed-Forces           -11.9062    563.204     -0.021      0.983   -1115.767    1091.954
onehot__Occupation_ Craft-repair            -0.0658      0.085     -0.773      0.439      -0.233       0.101
onehot__Occupation_ Exec-managerial          0.8027      0.082      9.846      0.000       0.643       0.962
onehot__Occupation_ Farming-fishing         -0.8591      0.150     -5.730      0.000      -1.153      -0.565
onehot__Occupation_ Handlers-cleaners       -0.8562      0.157     -5.442      0.000      -1.164      -0.548
onehot__Occupation_ Machine-op-inspct       -0.3711      0.109     -3.406      0.001      -0.585      -0.158
onehot__Occupation_ Other-service           -0.9055      0.124     -7.280      0.000      -1.149      -0.662
onehot__Occupation_ Priv-house-serv         -4.1290      1.723     -2.397      0.017      -7.506      -0.752
onehot__Occupation_ Prof-specialty           0.5292      0.085      6.254      0.000       0.363       0.695
onehot__Occupation_ Protective-serv          0.6215      0.136      4.567      0.000       0.355       0.888
onehot__Occupation_ Sales                    0.2664      0.088      3.044      0.002       0.095       0.438
onehot__Occupation_ Tech-support             0.5616      0.119      4.729      0.000       0.329       0.794
onehot__Occupation_ Transport-moving        -0.1427      0.106     -1.340      0.180      -0.351       0.066
onehot__Occupation_nan                      -1.6163   5.47e+05  -2.95e-06      1.000   -1.07e+06    1.07e+06
onehot__Relationship_Family                 -1.0682      0.149     -7.162      0

1. Workclass 'Without pay' is still innsignificant, will try to remove these instances (there is a small amount of them)
2. Some Occupations are insignifficant
3. Relationships became signifficant
4. All NaNs are insignifficant

Let's try to apply 'label encoder' to Workclass and Ocupation of 'one hot encoder'

# 4th model

In [118]:
categorical_features_list = ['Marital Status','Relationship', 'Ethnic group', 'Country', 'Sex']
numerical_features_list = ['Age', 'Capital Gain', 'Capital Loss', 'Hours per week', 'Occupation', 'Workclass']

In [119]:
data_clustered = data_train.copy()

le  = LabelEncoder()

data_clustered['Workclass'] = le.fit_transform(data_clustered['Workclass'])
data_clustered['Occupation'] = le.fit_transform(data_clustered['Occupation'])

data_clustered = cluster_categorical(data_clustered)

In [120]:
data_clustered

,Income,Age,Workclass,final weight,Education,Marital Status,Occupation,Relationship,Ethnic group,Sex,Capital Gain,Capital Loss,Hours per week,Country
23504,>50K,31,1,153005,Bachelors,Married,9,Family,White,Male,0,0,40,Developed
4116,<=50K,65,3,361721,Assoc-voc,Married,13,Family,White,Male,0,0,20,Developed
7027,<=50K,64,8,143716,Masters,Married,14,Family,White,Male,0,0,2,Developed
12918,>50K,61,8,198686,Assoc-acdm,Married,14,Family,White,Male,0,0,2,Developed
31261,<=50K,24,3,222853,Some-college,Single,2,Not-in-Family,White,Male,0,0,50,Developed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9176,<=50K,20,3,188612,Some-college,Single,0,Family,White,Female,0,0,38,Developing
15615,<=50K,36,3,188540,Bachelors,Single,3,Not-in-Family,White,Male,0,0,40,Developed
29900,>50K,38,3,91039,Masters,Married,3,Family,White,Male,0,0,65,Developed
22408,<=50K,34,3,422836,HS-grad,Single,9,Not-in-Family,White,Male,0,0,40,Developing


In [121]:
get_LR_performance(data_clustered, numerical_features_list, categorical_features_list)

f1 score: mean = 0.77 | std = 0.0
              precision    recall  f1-score   support

        >50K       0.87      0.93      0.90     19715
       <=50K       0.72      0.57      0.64      6333

    accuracy                           0.84     26048
   macro avg       0.80      0.75      0.77     26048
weighted avg       0.84      0.84      0.84     26048



In [122]:
summary = logit_summary(data_clustered, numerical_features_list, categorical_features_list)
summary

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.3393828778897463
            Iterations: 108
            Function evaluations: 111
            Gradient evaluations: 108


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           Income_ >50K   No. Observations:                26048
Model:                          Logit   Df Residuals:                    26031
Method:                           MLE   Df Model:                           16
Date:                Thu, 27 Apr 2023   Pseudo R-squ.:                  0.3881
Time:                        07:55:56   Log-Likelihood:                -8840.2
converged:                       True   LL-Null:                       -14448.
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================================
                                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
ordinal__Education                           0.3179      0.008     38.731      0.000       0.302       0.334
stand_scaler__Age                            0.3536      0.022     16.160      0.000       0.311       0.397
stand_scaler__Capital Gain                   2.2598      0.080     28.078      0.000       2.102       2.418
stand_scaler__Capital Loss                   0.2763      0.016     17.028      0.000       0.244       0.308
stand_scaler__Hours per week                 0.3701      0.021     18.010      0.000       0.330       0.410
stand_scaler__Occupation                     0.0111      0.020      0.561      0.575      -0.028       0.050
stand_scaler__Workclass                     -0.2156      0.021    -10.415      0.000      -0.256      -0.175
onehot__Marital Status_Single               -3.9491      0.147    -26.800      0.000      -4.238      -3.660
onehot__Relationship_Family                 -1.7252      0.135    -12.807      0.000      -1.989      -1.461
onehot__Relationship_Not-in-Family          -0.0615      0.164     -0.375      0.708      -0.383       0.260
onehot__Ethnic group_ Asian-Pac-Islander    -1.4729      0.181     -8.145      0.000      -1.827      -1.118
onehot__Ethnic group_ Black                 -1.8073      0.150    -12.011      0.000      -2.102      -1.512
onehot__Ethnic group_ Other                 -1.9690      0.308     -6.384      0.000      -2.574      -1.364
onehot__Ethnic group_ White                 -1.5595      0.134    -11.677      0.000      -1.821      -1.298
onehot__Country_Developing                  -0.6179      0.103     -5.991      0.000      -0.820      -0.416
onehot__Country_nan                         -0.2678      0.145     -1.844      0.065      -0.552       0.017
onehot__Sex_ Male                            0.0290      0.053      0.548      0.584      -0.075       0.133
============================================================================================================
"""

Ocupation feature is still insignificant, but the performance of model in total and especially for the minority class is now worse.
# 6th model

Let's try to remove missing data

In [123]:
data_no_nan = data_train.copy()
data_no_nan = data_no_nan.dropna(how='any')

In [124]:
data_no_nan.shape

(24107, 14)

In [125]:
numerical_features_list = ['Age', 'final weight', 'Capital Gain', 'Capital Loss', 'Hours per week']
categorical_features_list = ['Workclass', 'Marital Status', 'Occupation', 
                             'Relationship', 'Ethnic group', 'Sex', 'Country']

In [126]:
data_no_nan_clustered = cluster_categorical(data_no_nan)
data_no_nan_clustered


,Income,Age,Workclass,final weight,Education,Marital Status,Occupation,Relationship,Ethnic group,Sex,Capital Gain,Capital Loss,Hours per week,Country
23504,>50K,31,Local-gov,153005,Bachelors,Married,Prof-specialty,Family,White,Male,0,0,40,Developed
4116,<=50K,65,Private,361721,Assoc-voc,Married,Transport-moving,Family,White,Male,0,0,20,Developed
31261,<=50K,24,Private,222853,Some-college,Single,Craft-repair,Not-in-Family,White,Male,0,0,50,Developed
9271,<=50K,21,Private,72593,HS-grad,Single,Other-service,Family,White,Male,0,0,40,Developed
11241,<=50K,46,Private,288608,HS-grad,Single,Craft-repair,Family,White,Male,0,0,40,Developed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9176,<=50K,20,Private,188612,Some-college,Single,Adm-clerical,Family,White,Female,0,0,38,Developing
15615,<=50K,36,Private,188540,Bachelors,Single,Exec-managerial,Not-in-Family,White,Male,0,0,40,Developed
29900,>50K,38,Private,91039,Masters,Married,Exec-managerial,Family,White,Male,0,0,65,Developed
22408,<=50K,34,Private,422836,HS-grad,Single,Prof-specialty,Not-in-Family,White,Male,0,0,40,Developing


In [127]:
get_LR_performance(data_no_nan_clustered, numerical_features_list, categorical_features_list)

f1 score: mean = 0.79 | std = 0.0
              precision    recall  f1-score   support

        >50K       0.88      0.93      0.90     18050
       <=50K       0.74      0.61      0.67      6057

    accuracy                           0.85     24107
   macro avg       0.81      0.77      0.79     24107
weighted avg       0.84      0.85      0.84     24107



In [128]:
summary = logit_summary(data_no_nan_clustered, numerical_features_list, categorical_features_list)
summary

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.3329445111803958
            Iterations: 274
            Function evaluations: 277
            Gradient evaluations: 274


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           Income_ >50K   No. Observations:                24107
Model:                          Logit   Df Residuals:                    24073
Method:                           MLE   Df Model:                           33
Date:                Thu, 27 Apr 2023   Pseudo R-squ.:                  0.4094
Time:                        07:56:50   Log-Likelihood:                -8026.3
converged:                       True   LL-Null:                       -13589.
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================================
                                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
ordinal__Education                           0.2520      0.010     24.954      0.000       0.232       0.272
stand_scaler__Age                            0.3459      0.023     14.925      0.000       0.300       0.391
stand_scaler__final weight                   0.0765      0.020      3.775      0.000       0.037       0.116
stand_scaler__Capital Gain                   2.2864      0.085     26.749      0.000       2.119       2.454
stand_scaler__Capital Loss                   0.2641      0.017     15.345      0.000       0.230       0.298
stand_scaler__Hours per week                 0.3568      0.022     15.950      0.000       0.313       0.401
onehot__Workclass_ Local-gov                -0.9549      0.120     -7.941      0.000      -1.191      -0.719
onehot__Workclass_ Private                  -0.7590      0.099     -7.658      0.000      -0.953      -0.565
onehot__Workclass_ Self-emp-inc             -0.5586      0.135     -4.153      0.000      -0.822      -0.295
onehot__Workclass_ Self-emp-not-inc         -1.2436      0.118    -10.558      0.000      -1.474      -1.013
onehot__Workclass_ State-gov                -1.1105      0.135     -8.220      0.000      -1.375      -0.846
onehot__Workclass_ Without-pay             -24.1718   6.77e+04     -0.000      1.000   -1.33e+05    1.33e+05
onehot__Marital Status_Single               -3.5911      0.148    -24.212      0.000      -3.882      -3.300
onehot__Occupation_ Armed-Forces           -12.3652    710.862     -0.017      0.986   -1405.630    1380.899
onehot__Occupation_ Craft-repair            -0.1032      0.087     -1.187      0.235      -0.274       0.067
onehot__Occupation_ Exec-managerial          0.7264      0.083      8.711      0.000       0.563       0.890
onehot__Occupation_ Farming-fishing         -1.1261      0.155     -7.272      0.000      -1.430      -0.823
onehot__Occupation_ Handlers-cleaners       -0.8613      0.161     -5.334      0.000      -1.178      -0.545
onehot__Occupation_ Machine-op-inspct       -0.3787      0.111     -3.419      0.001      -0.596      -0.162
onehot__Occupation_ Other-service           -0.8585      0.127     -6.739      0.000      -1.108      -0.609
onehot__Occupation_ Priv-house-serv         -4.0684      1.726     -2.358      0.018      -7.450      -0.686
onehot__Occupation_ Prof-specialty           0.4813      0.087      5.542      0.000       0.311       0.652
onehot__Occupation_ Protective-serv          0.5479      0.139      3.949      0.000       0.276       0.820
onehot__Occupation_ Sales                    0.1960      0.090      2.185      0.029       0.020       0.372
onehot__Occupation_ Tech-support             0.5894      0.120      4.895      0.000       0.353       0.825
onehot__Occupation_ Transport-moving        -0.2443      0.109     -2.243      0.025      -0.458      -0.031
onehot__Relationship_Family                 -1.0181      0.153     -6.634      0

This approach to data preprocessing gave us the best result so far we saved computational complexity while redused the dimentionality, but the performance stayed. It's still not a good model though
# 7th model
### Let's try to apply ln() function to 'Age', 'Capital Gain' and 'Capital Loss' festures (as they are heavy tailed) before Standard Scaler to normalize it

In [74]:
data_logged = data_train.copy()
data_logged = data_logged.dropna(how='any')

In [75]:
data_logged['Capital Gain'] = np.log(1+ data_logged['Capital Gain'])
data_logged['Capital Loss'] = np.log(1+ data_logged['Capital Loss'])
data_logged['Age'] = np.log(data_logged['Age'])

data_logged = cluster_categorical(data_logged)

In [76]:
get_LR_performance(data_logged, numerical_features_list, categorical_features_list)

/Users/nadiiaduiunova/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/nadiiaduiunova/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/Users/nadiiaduiunova/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 276, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/Users/nadiiaduiunova/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/Users/nadiiaduiunova/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/pipeline.py", line 480, in predict
    Xt = 

f1 score: mean = nan | std = nan
              precision    recall  f1-score   support

        >50K       0.88      0.93      0.90     18105
       <=50K       0.73      0.61      0.67      5996

    accuracy                           0.85     24101
   macro avg       0.80      0.77      0.78     24101
weighted avg       0.84      0.85      0.84     24101



Not better either.
# 8th model
### Another try is to cluster 'Hours per week' feature to part-time, fulltime and overtime workers with fulltime value for 40 hours

In [77]:
data_new = data_train.copy()
data_new = data_new.dropna(how='any')
data_new['Hours per week'] = np.where(data_new['Hours per week'] == 40, 'fulltime', 
                                   (np.where(data_new['Hours per week'] < 40, 'part-time', 'overtime')))

data_new = cluster_categorical(data_new)

data_new = data_new.drop(['final weight'], axis='columns')

In [78]:
data_new.head()

,Income,Age,Workclass,Education,Marital Status,Occupation,Relationship,Ethnic group,Sex,Capital Gain,Capital Loss,Hours per week,Country
14793,>50K,44,Private,Bachelors,Married,Exec-managerial,Husband,Black,Male,0,0,fulltime,Developed
12035,<=50K,65,Self-emp-not-inc,7th-8th,Married,Farming-fishing,Husband,White,Male,0,0,fulltime,Developed
28658,>50K,47,Private,Bachelors,Married,Sales,Husband,White,Male,0,0,overtime,Developed
16127,<=50K,43,Private,HS-grad,Married,Machine-op-inspct,Husband,White,Male,0,0,fulltime,Developed
29763,>50K,41,Private,Prof-school,Married,Exec-managerial,Husband,White,Male,0,0,fulltime,Developed


In [79]:
categorical_features_list = ['Workclass', 'Marital Status', 'Occupation', 
                             'Relationship', 'Ethnic group', 'Sex', 'Country', 'Hours per week']
numerical_features_list = ['Age', 'Capital Gain', 'Capital Loss']

In [80]:
get_LR_performance(data_new, numerical_features_list, categorical_features_list)

f1 score: mean = 0.79 | std = 0.01
              precision    recall  f1-score   support

        >50K       0.88      0.93      0.90     18105
       <=50K       0.74      0.61      0.67      5996

    accuracy                           0.85     24101
   macro avg       0.81      0.77      0.79     24101
weighted avg       0.84      0.85      0.84     24101



In [81]:
summary = logit_summary(data_new, numerical_features_list, categorical_features_list)
summary

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.3271185756280581
            Iterations: 294
            Function evaluations: 296
            Gradient evaluations: 294


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           Income_ >50K   No. Observations:                24101
Model:                          Logit   Df Residuals:                    24064
Method:                           MLE   Df Model:                           36
Date:                Thu, 27 Apr 2023   Pseudo R-squ.:                  0.4169
Time:                        07:44:42   Log-Likelihood:                -7883.9
converged:                       True   LL-Null:                       -13521.
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================================
                                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
ordinal__Education                           0.2128      0.010     21.809      0.000       0.194       0.232
stand_scaler__Age                            0.3475      0.024     14.641      0.000       0.301       0.394
stand_scaler__Capital Gain                   2.4476      0.090     27.223      0.000       2.271       2.624
stand_scaler__Capital Loss                   0.2585      0.017     14.981      0.000       0.225       0.292
onehot__Workclass_ Local-gov                -1.2447      0.120    -10.377      0.000      -1.480      -1.010
onehot__Workclass_ Private                  -1.0719      0.097    -11.034      0.000      -1.262      -0.881
onehot__Workclass_ Self-emp-inc             -0.7041      0.134     -5.256      0.000      -0.967      -0.442
onehot__Workclass_ Self-emp-not-inc         -1.5177      0.118    -12.886      0.000      -1.749      -1.287
onehot__Workclass_ State-gov                -1.3714      0.135    -10.127      0.000      -1.637      -1.106
onehot__Workclass_ Without-pay              -4.4955      3.430     -1.311      0.190     -11.218       2.227
onehot__Marital Status_Single               -2.5399      0.281     -9.045      0.000      -3.090      -1.990
onehot__Occupation_ Armed-Forces            -2.5719      1.371     -1.876      0.061      -5.259       0.115
onehot__Occupation_ Craft-repair            -0.1863      0.086     -2.155      0.031      -0.356      -0.017
onehot__Occupation_ Exec-managerial          0.6476      0.084      7.720      0.000       0.483       0.812
onehot__Occupation_ Farming-fishing         -1.2050      0.155     -7.753      0.000      -1.510      -0.900
onehot__Occupation_ Handlers-cleaners       -0.9394      0.161     -5.853      0.000      -1.254      -0.625
onehot__Occupation_ Machine-op-inspct       -0.6285      0.113     -5.560      0.000      -0.850      -0.407
onehot__Occupation_ Other-service           -0.9848      0.131     -7.514      0.000      -1.242      -0.728
onehot__Occupation_ Priv-house-serv         -3.6673      1.587     -2.311      0.021      -6.777      -0.557
onehot__Occupation_ Prof-specialty           0.5661      0.088      6.434      0.000       0.394       0.739
onehot__Occupation_ Protective-serv          0.4683      0.140      3.353      0.001       0.195       0.742
onehot__Occupation_ Sales                    0.1243      0.090      1.376      0.169      -0.053       0.301
onehot__Occupation_ Tech-support             0.5494      0.122      4.498      0.000       0.310       0.789
onehot__Occupation_ Transport-moving        -0.3742      0.108     -3.451      0.001      -0.587      -0.162
onehot__Relationship_ Not-in-family          0.4434      0.286      1.553      0.120      -0.116       1.003
onehot__Relationship_ Other-relative        -0.4526      0.255     -1.774      0.076      -0.953       0.048
onehot__Relationship_ Own-child             -0.7326      0.286     -2.562      0

# 9th model
### Lets now try to cluster all minority categories of imbalanced features together

In [ ]:
X_cluster2 = X_train.copy()
def balance_predictors(X):
    X['Ethnic group'] = np.where(X['Ethnic group'] != ' White', 'Other', X['Ethnic group'])
    X['Country'] = np.where(X['Country'] != ' United-States', 'Other', X['Country'])
    X['Workclass'] = np.where(X['Workclass'] != ' Private', 'Other', X['Workclass'])
    X['Marital Status'] = np.where(((X['Marital Status'] == ' Widowed') |
                                    (X['Marital Status'] == ' Married-spouse-absent') |
                                    (X['Marital Status'] == ' Separated')), 
                                    'Other', X_train['Marital Status'])
    X['Occupation'] = np.where(((X['Occupation'] == ' Adm-clerical') |
                                (X['Occupation'] == ' Armed-Forces') |
                                (X['Occupation'] == ' Craft-repair') |
                                (X['Occupation'] == ' Machine-op-inspct') |
                                (X['Occupation'] == ' Priv-house-serv') |
                                (X['Occupation'] == ' Transport-moving')), 
                                'Other', X['Occupation'])
    X['Hours per week'] = np.where(X['Hours per week'] == 40, 'fulltime', (np.where(X['Hours per week'] < 40, 'part-time', 'overtime')))
    
balance_predictors(X_cluster2)
X_cluster2.sample(3)

In [ ]:
numerical_features_list = ['Age', 'Capital Gain', 'Capital Loss']
categorical_features_list = ['Workclass', 'Marital Status', 'Occupation', 
                             'Relationship', 'Ethnic group', 'Sex', 'Country', 'Hours per week']

In [ ]:
get_LR_performance(X_cluster2, y_train, numerical_features_list, categorical_features_list)

# Now let's try to find the threshold for optimal recall and precisoin values as the dataset is imbalanced and predicts minority class much worse so far

In [ ]:
X_10 = X_train.copy()
y_10 = y_train.copy()
X_10.sample()

In [ ]:
# preform data transformation as we used for initial model

categorical_features_list = ['Workclass', 'Marital Status', 'Occupation', 
                             'Relationship', 'Ethnic group', 'Sex', 'Country']
numerical_features_list = ['Age', 'final weight', 'Capital Gain', 'Capital Loss', 'Hours per week']

cluster_categorical(X_10)

column_transformer = ColumnTransformer(transformers = [
    ('ordinal', OrdinalEncoder(categories=[[' Preschool',' 1st-4th',' 5th-6th',' 7th-8th',' 9th',' 10th',' 11th',
                                        ' 12th',' HS-grad',' Some-college',' Assoc-voc',' Assoc-acdm', 
                                        ' Bachelors',' Masters',' Prof-school',' Doctorate']]),
        make_column_selector(pattern = 'Education')),
    ('minmax_scaler', MinMaxScaler(), numerical_features_list),
    ('onehot', OneHotEncoder(dtype='int', drop='first'), categorical_features_list)],
    remainder='drop')

X_10 = column_transformer.fit_transform(X_10)

if sps.issparse(X_10):
    X_10 = X_10.toarray()
    
x_columns_names = column_transformer.get_feature_names_out()
X_10 = pd.DataFrame(X_10, columns = x_columns_names)

In [ ]:
X_10['ordinal__Education'] = MinMaxScaler().fit_transform(X_10[['ordinal__Education']])
X_10.sample()

In [ ]:
y_10 = y_10.replace({' >50K': 1, ' <=50K': 0})
y_10.sample(5)

In [ ]:
# Apply Stochastic Gradient Descent to find global optimum  of the cost function

sgd_clf = SGDClassifier(loss = 'modified_huber')
sgd_clf.fit(X_10, y_10)

In [ ]:
print(sgd_clf.predict([X_10.iloc[2]]), y_10.iloc[2])

In [ ]:
cross_val_score(sgd_clf, X_10, y_10, cv=5, scoring="f1_macro")

In [ ]:
y_10_pred_sgd = cross_val_predict(sgd_clf, X_10, y_10, cv=3)
confusion_matrix(y_10, y_10_pred_sgd)

In [ ]:
print(classification_report(y_10, y_10_pred_sgd))

In [ ]:
# transform test data to check

cluster_categorical(X_test)

column_transformer1 = ColumnTransformer(transformers = [
    ('ordinal', OrdinalEncoder(categories=[[' Preschool',' 1st-4th',' 5th-6th',' 7th-8th',' 9th',' 10th',' 11th',
                                        ' 12th',' HS-grad',' Some-college',' Assoc-voc',' Assoc-acdm', 
                                        ' Bachelors',' Masters',' Prof-school',' Doctorate']]),
        make_column_selector(pattern = 'Education')),
    ('minmax_scaler', MinMaxScaler(), numerical_features_list),
    ('onehot', OneHotEncoder(dtype='int', drop='first'), categorical_features_list)],
    remainder='drop')

X_test = column_transformer1.fit_transform(X_test)

if sps.issparse(X_test):
    X_test = X_test.toarray()
    
x_columns_names = column_transformer1.get_feature_names_out()
X_test = pd.DataFrame(X_test, columns = x_columns_names)

y_test = y_test.replace({' >50K': 1, ' <=50K': 0})
y_test.sample(5)

In [ ]:
y_test_pred_sgd = cross_val_predict(sgd_clf, X_test, y_test, cv=3)
print(classification_report(y_test, y_test_pred_sgd))

More than 3k of false negatives (minority class in this case), which is pretty bad

In [ ]:
y_10_scores_sgd = sgd_clf.decision_function([X_10.iloc[90]])
y_10_scores_sgd

In [ ]:
y_10_scores_sgd = cross_val_predict(sgd_clf, X_10, y_10, cv=3, method="decision_function")
precisions_sgd, recalls_sgd, thresholds_sgd = precision_recall_curve(y_10, y_10_scores_sgd)

In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds): 
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision") 
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall") 
    plt.xlabel("Threshold")
    plt.legend(loc="upper left")
    plt.ylim([0, 1])


plot_precision_recall_vs_threshold(precisions_sgd, recalls_sgd, thresholds_sgd)
plt.show()

In [ ]:
def plot_roc_curve(fpr, tpr, label=None): 
    plt.plot(fpr, tpr, linewidth=2, label=label) 
    plt.plot([0, 1], [0, 1], 'k--') 
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')

fpr_sgd, tpr_sgd, thresholds_sgd = roc_curve(y_10, y_10_scores_sgd)
plot_roc_curve(fpr_sgd, tpr_sgd)
plt.show()

## Let's now compare it to Logistic Regression 

In [ ]:
lg_clf = LogisticRegression(max_iter=500, C = 100)
lg_clf.fit(X_10, y_10)

In [ ]:
print(lg_clf.predict([X_10.iloc[2]]), y_10.iloc[2])

In [ ]:
cross_val_score(lg_clf, X_10, y_10, cv=3, scoring="f1_macro")

In [ ]:
y_10_pred_lg = cross_val_predict(lg_clf, X_10, y_10, cv=3)
confusion_matrix(y_10, y_10_pred_lg)

In [ ]:
print(classification_report(y_10, y_10_pred_lg))

In [ ]:
y_10_scores_lg = lg_clf.decision_function([X_10.iloc[90]])
y_10_scores_lg

In [ ]:
y_10_scores_lg = cross_val_predict(lg_clf, X_10, y_10, cv=3, method="decision_function")
precisions_lg, recalls_lg, thresholds_lg = precision_recall_curve(y_10, y_10_scores_lg)

In [ ]:
plot_precision_recall_vs_threshold(precisions_lg, recalls_lg, thresholds_lg)
plt.show()

In [ ]:
fpr_lg, tpr_lg, thresholds_lg = roc_curve(y_10, y_10_scores_lg)

In [ ]:
plot_roc_curve(fpr_lg, tpr_lg)
plt.show()

# Support Vector Machine classifier with polynomial kernel

In [ ]:
poly_kernel_svm_clf = SVC(C = 10, gamma = 0.1, kernel = 'rbf')

In [ ]:
poly_kernel_svm_clf.fit(X_10, y_10)

In [ ]:
y_10_pred_svm = cross_val_predict(poly_kernel_svm_clf, X_10, y_10, cv=3)

In [ ]:
confusion_matrix(y_10, y_10_pred_svm)

In [ ]:
print(classification_report(y_10, y_10_pred_svm))

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'kernel': ['rbf'], 'gamma': [1, 0.1, 0.01], 'C': [1, 10, 100]},
    {'kernel': ['poly'], 'degree': [7, 14, 28], 'coef0': [0.1, 1,10], 'C': [1, 10, 100]},
]
svm = SVC()
grid_search = GridSearchCV(svm, param_grid, cv=5, refit = True)
grid_search.fit(X_10, y_10)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
pca = PCA(n_components=37)
X2D = pca.fit_transform(X_10)

In [ ]:
y_10_pred_pca = cross_val_predict(LogisticRegression(), X2D, y_10, cv=3)

In [ ]:
print(classification_report(y_10, y_10_pred_pca))

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
X_10

In [ ]:
from sklearn.model_selection import GridSearchCV 
from sklearn.linear_model import LogisticRegression 
from sklearn.pipeline import Pipeline

from sklearn.decomposition import KernelPCA

clf = Pipeline([
        ("kpca", KernelPCA(n_components=2)),
        ("log_reg", LogisticRegression())
])
param_grid = [{
        "kpca__gamma": np.linspace(0.03, 0.05, 10),
        "kpca__kernel": ["rbf", "sigmoid"]
        }]
grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X_10, y_10)

print(grid_search.best_params_)